In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
import string

TRUE_CUTOFF = 500

In [2]:
replace_char = ["*", "&", "%", "/", "\\"]
strip_char = ["'", "-", ".", "!", ":", ";"]
num_char =  ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"]
common_errors = {
    "0": "O",
    "1": "l",
    "5": "S",
    "8": "B"
}

In [3]:
map_dict = {}

In [4]:
def clean_header(h):
    for s in replace_char:
        h = h.replace(s, "")
    for s in strip_char:
        h = h.strip(s)
    cnt = 0
    hl = []
    for c in list(h):
        if c in common_errors: c = common_errors[c]
        if c in num_char: cnt += 1
        hl.append(c)
    h = string.join(hl, "")
    if cnt > 4: h = ""
    return h.upper()
    
def assign_clean(D):
    return [clean_header(h) for h in D.true_headers]

def score(string1, string2):
    # Scores the fuzzy match between the true header and the ocr header
    return fuzz.partial_ratio(string1,string2)

def match(headers, true_headers, map_dict):
    # Matches ocr headers to true headers
    for header in headers:
        if header not in map_dict:
            score_list = [(score(header, true_header), true_header) for true_header in true_headers]
            sorted_score = sorted(score_list, key = lambda tup: tup[0])
            score_tuple = sorted_score[0]
            map_dict[header] = score_tuple
    return map_dict

In [5]:
raw = pd.read_csv("RankedHeaders.csv")[["count", "true_headers"]].dropna().assign(true_headers=assign_clean)

In [6]:
df = raw.drop_duplicates("true_headers")
df = df[df.true_headers.map(lambda h: (len(h) < 150) and (len(h) > 2) and (h is not ""))].reset_index(drop=True)
true_headers = list(df[:TRUE_CUTOFF].true_headers)
unsure_headers = list(df.true_headers)

In [7]:
map_dict = match(unsure_headers, true_headers, map_dict)

In [8]:
map_dict

{'ORG AN BUILDERS': (13, 'SLITTING'),
 'R.EAL ESTATE AGENTS': (14, 'SCHOOLS-PUBLIC'),
 'PEWTER WARE MFRS': (0, 'ING'),
 "HEARING AIDS-CON'I'D": (8, 'UPHOLSTERERS'),
 'INVESTMENT BROKERS': (14, 'CLINICS'),
 'ELECTRICAL APPLIANCES SALES AN D SERVICE': (22, 'IMPORTERS'),
 'CORDAGE AND TWINE-MTRS': (7, 'SCHOOLS-PUBLIC'),
 'LITHOGRAPHERS AN D BRIEF PRINTERS': (17, 'CLUBS-SOCIAL'),
 'PUBLISHERS-BOOK, NEWSPAPER  PERIODICALS': (20, 'DYE STUFFS'),
 'PREUERS-HOME FREEZERS': (0, 'ING'),
 'BUS-ES FOR HIRE': (13, 'SLITTING'),
 'BILLIARD ROOM EQUIPMENT AND SUPPLIES': (18, 'CLUBS-CIVIC'),
 'INSURANCE,-C ASUALTY': (17, 'JEWELERS-MFG'),
 'SCHOOLS  ACADEMIES': (13, 'FURNITURE-RETAIL'),
 "CLOTHING MFRS- WOMEN'S AN D MISSES": (17, 'SALARY'),
 'MUSICAL INSTRUMEN T MFRS': (20, 'BATHS'),
 'RUBBER TIRDS  QUBE;  SEAL STAMPS-MFRS  DEALERS': (20, 'LAW OFFICB'),
 'REST HOMES': (0, 'ING'),
 'WATER BEDS': (0, 'ING'),
 'WIRE MANUFACTURE RS': (12, 'SCHOOLS-PAROCHIAL'),
 'COMMUNITY CENTERS': (13, 'HARDWARE-RETAIL'),
 